In [1]:
%reset -f 
import scipy
import paddle
from numpy import trace as np_trace
import paddle_quantum as pq
from paddle_quantum.ansatz import Circuit
from paddle_quantum.state import zero_state
from paddle_quantum.qinfo import state_fidelity, partial_trace, pauli_str_to_matrix

)


As a hands-on example, here we consider a 3-qubit Hamiltonian and its Gibbs state:

$$
H = -Z \otimes Z \otimes I - I \otimes Z \otimes Z - Z \otimes I \otimes Z, \quad I=\left [
\begin{matrix}
1 & 0  \\
0 & 1  \\
\end{matrix} 
\right ], \quad 
Z=\left [
\begin{matrix}
1 & 0  \\
0 & -1  \\
\end{matrix} 
\right ].
\tag{2}
$$

In this example, we set the inverse temperature parameter to $\beta = 1.5$. Besides, to test the final results, we have generated the ideal Gibbs state $\rho_G$ in advance according to the definition.

In [2]:
# N = 12                               # The width of the QNN
N_SYS_B = 3                         # The number of qubits of subsystem B used to generate the Gibbs state
SEED = 16                           # Fixed random seed
beta = 1.5                          # Set the inverse temperature parameter beta
pq.set_backend('density_matrix')    # Set density matrix backend
pq.set_dtype('complex128')          # Set calculation precision

In [3]:
import paddle
import paddle_quantum as pq
from paddle_quantum import Hamiltonian
from paddle_quantum.state import ghz_state
from paddle_quantum.qinfo import random_pauli_str_generator

# # Generate a specific Hamiltonian represented by a Pauli string
# H = [[-1.0,'z0,z1'], [-1.0,'z1,z2'], [-1.0,'z0,z2'], [-1.0,'y0,y1']]
# print (Hamiltonian(H))
# # Generate matrix information of Hamiltonian
# hamiltonian = pauli_str_to_matrix(H, N_SYS_B).numpy()

def randHamil(terms=10):    
    pauli_str = random_pauli_str_generator(N_SYS_B, terms=terms)
    hamiltonian = Hamiltonian(pauli_str)
#     print(hamiltonian)
    hamiltonian = paddle.to_tensor(hamiltonian.construct_h_matrix()).numpy()
    # print (H_matrix)

    # Generate the ideal target Gibbs state rho
    rho_G = scipy.linalg.expm(-1 * beta * hamiltonian) / np_trace(scipy.linalg.expm(-1 * beta * hamiltonian))

    # print (rho_G)
    # Set to the data type supported by Paddle Quantum
    hamiltonian = hamiltonian.astype('complex128')
    # print (hamiltonian)
    rho_G = paddle.to_tensor(rho_G, dtype='complex128')
    
    return hamiltonian, rho_G

In [4]:
def U_theta(num_qubits: int, depth: int) -> Circuit:
    """
    Quantum Neural Network
    """
    # Initialize the QNN according to the number of qubits
    cir = Circuit(num_qubits)
    
    # Built-in {R_y + CNOT} circuit template
    cir.real_entangled_layer(depth=depth)    
    # The QNN acts on a given initial state
    cir.ry()   
    cir.real_entangled_layer(depth=depth)    
    cir.rx()
    return cir

### Configuring the training model: loss function

- Now that we have the data and QNN architecture, we also need to define appropriate training parameters, models, and loss functions to achieve our goals.

- Specifically, we refer to the method in the paper [4]. The core idea is to use the Gibbs state to achieve the minimum free energy.

- By applying the QNN $U(\theta)$ on the initial state, we can get the output state $\left| {\psi \left( {\bf{\theta }} \right)} \right\rangle $. Its state in the 2-4th qubits is recorded as $\rho_B(\theta)$.

- Set the loss function in the training model. In Gibbs state learning, we use the truncation of the von Neumann entropy function to estimate the free energy, and the corresponding loss function, as in reference [4], can be set as $loss = {L_1} + {L_2} + {L_3} $, where

$$
{L_1}= \text{tr}(H\rho_B), \quad {L_2} = 2{\beta^{-1}}{\text{tr}}(\rho_B^2), \quad L_3 = - {\beta ^{ - 1}}\big(\text{tr}(\rho_B^3) + 3\big)/2.
\tag{3}
$$

In [5]:
 # define loss function
def loss_func(cir: Circuit, Hamiltonian: paddle.Tensor, N_SYS_B: int, N_B: int) -> paddle.Tensor:
    # Apply QNN
    rho_AB = cir(zero_state(N_B))
    
    # Calculate partial trace to obtain the quantum state rho_B of subsystem B
    rho_B = partial_trace(rho_AB.data, 2 ** (N_B - N_SYS_B), 2 ** (N_SYS_B), 1)
    
    # Calculate the three parts of the loss function
    rho_B_squre = rho_B @ rho_B
    loss1 = paddle.real(paddle.trace(rho_B @ Hamiltonian))
    loss2 = paddle.real(paddle.trace(rho_B_squre)) * 2 / beta
    loss3 = -(paddle.real(paddle.trace(rho_B_squre @ rho_B)) + 3) / (2 * beta)
    
    # 最终的损失函数
    loss = loss1 + loss2 + loss3  
    
    return loss, rho_B

### Configure training model: model parameters

Before training the QNN, we also need to set some training hyperparameters, mainly the learning rate (LR), the number of iterations (ITR), and the depth (D) of the QNN. Here we set the learning rate to 0.5 and the number of iterations to 50. Readers may wish to adjust by themselves to explore the influence of hyperparameter adjustment on the training effect.

In [6]:
ITR = 100 # Set the total number of iterations of training
LR = 0.3 # Set the learning rate
D = 1 # Set the depth of the QNN

### Training

- After all the training model parameters are set, we convert the data into Tensor in PaddlePaddle and then train the QNN.
- During training, we use [Adam Optimizer](https://www.paddlepaddle.org.cn/documentation/docs/en/api/paddle/optimizer/adam/Adam_en.html). Other optimizers are also provided in PaddlePaddle.
- We output the results of the training process in turn.
- In particular, we sequentially output the fidelity of the quantum state $\rho_B(\theta)$ and Gibbs state $\rho_G$ we learned. The higher the fidelity, the closer the QNN output state is to Gibbs state.

In [ ]:
import tqdm 
    
# Convert Numpy array to Tensor supported in PaddlePaddle

for k in tqdm.tqdm([8,10,12,14]):
    paddle.seed(SEED)
    hamiltonian, rho_G= randHamil(terms=k)
    H = paddle.to_tensor(hamiltonian)    
    for i in tqdm.tqdm([4,6,8]):   
        single_cir=U_theta(int(i), D)        
        # Generally speaking, we use Adam optimizer to get relatively good convergence
        # Of course, it can be changed to SGD or RMS prop.
        opt = paddle.optimizer.Adam(learning_rate=LR, parameters=single_cir.parameters())

        # Optimization loops
        for itr in (range(1, ITR + 1)):

            # Run forward propagation to calculate the loss function and return the generated quantum state rho_B
            loss, rho_B = loss_func(single_cir, H, N_SYS_B, i)

            # Run back propagation to minimize the loss function
            loss.backward()
            opt.minimize(loss)
            opt.clear_grad()

            # Convert to Numpy array to calculate the fidelity of the quantum state F(rho_B, rho_G)
            fid = state_fidelity(rho_B, rho_G)

            # Print training results
#             if itr % 10 == 0:
#                 print('iter:', itr, 'loss:', '%.4f' % loss.numpy(), 'fid:', '%.4f' % fid.numpy())
            if itr == ITR:                
#                 print(single_cir)
                print('Q:',i, 'iter:', itr, 'loss:', '%.4f' % loss.numpy(), 'fid:', '%.4f' % fid.numpy())

  0%|                                                                                             | 0/3 [00:00<?, ?it/s]/opt/conda/lib/python3.8/site-packages/paddle/fluid/framework.py:1104: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  elif dtype == np.bool:
/opt/conda/lib/python3.8/site-packages/paddle/tensor/creation.py:125: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if data.dtype == np.object:

 33%|████

Q: 4 iter: 100 loss: -2.5748 fid: 0.8793



 67%|████████████████████████████████████████████████████████▋                            | 2/3 [00:14<00:07,  7.84s/it]

Q: 6 iter: 100 loss: -2.5382 fid: 0.8848


## Conclusion

According to the results obtained from the above training, after about 50 iterations, we can achieve a high-precision Gibbs state with a fidelity higher than 99.5% and complete the preparation of the Gibbs state efficiently and accurately. We can output the QNN's learned parameters and its output state through the print function.

_______

## References

[1] Kieferová, M. & Wiebe, N. Tomography and generative training with quantum Boltzmann machines. [Phys. Rev. A 96, 062327 (2017).](https://journals.aps.org/pra/abstract/10.1103/PhysRevA.96.062327)

[2] Brandao, F. G. S. L. & Svore, K. M. Quantum Speed-Ups for Solving Semidefinite Programs. [in 2017 IEEE 58th Annual Symposium on Foundations of Computer Science (FOCS) 415–426 (IEEE, 2017). ](https://ieeexplore.ieee.org/abstract/document/8104077)

[3] Somma, R. D., Boixo, S., Barnum, H. & Knill, E. Quantum Simulations of Classical Annealing Processes. [Phys. Rev. Lett. 101, 130504 (2008).](https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.101.130504)

[4] Wang, Y., Li, G. & Wang, X. Variational quantum Gibbs state preparation with a truncated Taylor series. [Phys. Rev. A 16, 054035 (2021).](https://journals.aps.org/prapplied/abstract/10.1103/PhysRevApplied.16.054035)